In [1]:
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
from collections import deque
import torch
from torch import nn
from DeepQLearning import DeepQLearning
from learning import Agent

env = gym.make('MountainCar')
#env.seed(0)
np.random.seed(0)

## pytorch model
model = nn.Sequential(
            nn.Linear(env.observation_space.shape[0],512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,env.action_space.n)
        )

criterion = nn.MSELoss()# MSE
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)# ADAM


d_params = {
    "env":env,
    "gamma" : 0.99,
    "epsilon" : 1.0,
    "epsilon_min" : 0.01,
    "epsilon_dec" : 0.99,
    "episodes" : 400,
    "batch_size" : 128,
    "memory" : deque(maxlen=10000), #talvez usar uma memoria mais curta
    "model" : model,
    "criterion" : criterion,
    "optimizer" : optimizer,
    "max_steps" : 500,
}


q_params = {
    "epsilon":0.9,
    "alpha":0.1,
    "gamma":0.99,
    "e_dec":0.999,
    "e_min":0.01,
    "algo":"q-learning",
    "env":"MountainCar-v0",
    "cont":True,
    "training":False,
    "max_episodes": 400}






/home/andre/miniconda3/envs/cuda/lib/python3.8/site-packages/gymnasium/envs/registration.py:527: UserWarning: WARN: Using the latest versioned environment `MountainCar-v0` instead of the unversioned environment `MountainCar`.
  logger.warn(


In [2]:
def rolling(rewards,window):
    for i in range(window,len(rewards)):
         rewards[i-window:i].mean()
    return rewards

In [6]:
q_list = []
for i in range(5):
    Q = Agent(**q_params)
    Q.read_training("data/q_learning.txt",inplace=True)
    rewards = np.array(Q.execute())
    q_list.append(rolling(rewards,50))
    
    print(f"run {i}")

run 0
run 1
run 2
run 3
run 4


In [7]:
print(q_list[0])

[-163. -163. -162. -165. -163. -159. -151. -165. -154. -156. -158. -157.
 -156. -157. -162. -158. -164. -159. -162. -155. -164. -151. -157. -157.
 -157. -163. -159. -162. -152. -165. -161. -148. -161. -151. -161. -158.
 -158. -155. -163. -164. -156. -155. -164. -167. -160. -164. -156. -155.
 -163. -163. -157. -165. -159. -156. -156. -164. -152. -161. -159. -163.
 -162. -158. -164. -159. -160. -166. -166. -150. -160. -157. -156. -148.
 -152. -164. -160. -163. -157. -156. -151. -154. -159. -163. -156. -167.
 -157. -165. -152. -146. -152. -163. -158. -161. -153. -166. -162. -152.
 -162. -161. -158. -157. -163. -160. -166. -155. -155. -156. -160. -162.
 -161. -163. -162. -166. -160. -167. -158. -158. -156. -156. -151. -158.
 -164. -151. -160. -160. -162. -157. -160. -155. -162. -161. -162. -151.
 -167. -157. -165. -164. -155. -163. -163. -162. -162. -163. -163. -161.
 -166. -162. -160. -160. -164. -160. -160. -165. -151. -162. -161. -164.
 -152. -162. -152. -163. -163. -162. -154. -159. -1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def prepare_data_for_seaborn(d_list, q_list):
    all_data = []
    
    for run_idx, run_data in enumerate(d_list):
        for step_idx, reward in enumerate(run_data):
            all_data.append({
                'Algorithm': 'DQN',
                'Run': run_idx,
                'Step': step_idx,
                'Reward': reward
            })
    
    for run_idx, run_data in enumerate(q_list):
        for step_idx, reward in enumerate(run_data):
            all_data.append({
                'Algorithm': 'Q-learning',
                'Run': run_idx,
                'Step': step_idx,
                'Reward': reward
            })
    
    return pd.DataFrame(all_data)

sns.set(style="darkgrid")
plt.figure(figsize=(12, 6))

df = prepare_data_for_seaborn(d_list, q_list)

sns.lineplot(
    data=df,
    x='Step',
    y='Reward',
    hue='Algorithm',
    errorbar=('ci', 95),
    palette=['blue', 'red']
)

plt.title('Training Curves: DQN vs Q-learning with 95% CI', fontsize=16)
plt.xlabel('Training Steps (Rolling Window = 50)', fontsize=14)
plt.ylabel('Average Reward', fontsize=14)
plt.legend(title='Algorithm')

plt.tight_layout()
plt.show()